In [1]:
import os
import pandas as pd
pd.options.display.max_columns = 250

import machine_learning as ml
from preprocessing import separate_features_target

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [2]:
# Path
cwd = os.getcwd()
csv_path = os.path.join(cwd, 'data/csv/')

train = {
    'stylometric' : ['style_train_balanced.csv','style_train_imbalanced.csv'],
    'word2vec' : ['word2vec_train_balanced.csv','word2vec_train_imbalanced.csv']
}
test = {
    'stylometric' : ['style_test_balanced.csv','style_test_imbalanced.csv'],
    'word2vec' : ['word2vec_test_balanced.csv','word2vec_test_imbalanced.csv']
}

## Import Data

Since Word2Vec features outperformed the TF-IDF features, only those will be used to test the combination with content features.

### Balanced Dataset

In [3]:
style_train_balanced_complete = pd.read_csv(os.path.join(csv_path, train['stylometric'][0]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})
style_test_balanced_complete = pd.read_csv(os.path.join(csv_path, test['stylometric'][0]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})

word2vec_train_balanced_complete = pd.read_csv(os.path.join(csv_path, train['word2vec'][0]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})
word2vec_test_balanced_complete = pd.read_csv(os.path.join(csv_path, test['word2vec'][0]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})

In [4]:
style_train_balanced = separate_features_target(style_train_balanced_complete)
style_test_balanced = separate_features_target(style_test_balanced_complete)

word2vec_train_balanced = separate_features_target(word2vec_train_balanced_complete)
word2vec_test_balanced = separate_features_target(word2vec_test_balanced_complete)

### Imbalanced Dataset

In [5]:
style_train_imbalanced_complete = pd.read_csv(os.path.join(csv_path, train['stylometric'][1]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})
style_test_imbalanced_complete = pd.read_csv(os.path.join(csv_path, test['stylometric'][1]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})

word2vec_train_imbalanced_complete = pd.read_csv(os.path.join(csv_path, train['word2vec'][1]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})
word2vec_test_imbalanced_complete = pd.read_csv(os.path.join(csv_path, test['word2vec'][1]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})

In [6]:
style_train_imbalanced = separate_features_target(style_train_imbalanced_complete)
style_test_imbalanced = separate_features_target(style_test_imbalanced_complete)

word2vec_train_imbalanced = separate_features_target(word2vec_train_imbalanced_complete)
word2vec_test_imbalanced = separate_features_target(word2vec_test_imbalanced_complete)

# Merging Feature Sets

The simplest way of combining the information of the two different feature sets is to simply merge them into one set and then perform the predictions based on this concatenated set.

## Balanced Dataset

In [7]:
style_content_train_balanced = pd.concat([word2vec_train_balanced['features'], style_train_balanced['features']], axis=1)
style_content_test_balanced = pd.concat([word2vec_test_balanced['features'], style_test_balanced['features']], axis=1)

### Train

#### Logistic Regression

In [8]:
lr_style_content_balanced = ml.train_logistic_regression(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1)
lr_style_content_balanced, lr_style_content_balanced_scaler = lr_style_content_balanced['model'], lr_style_content_balanced['scaler']

Train accuracy: 0.9944320712694877


#### Decision Tree

In [9]:
dt_style_content_balanced = ml.train_decision_tree(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1)

Train accuracy: 0.9862657757980697


#### Random Forest

In [10]:
rf_style_content_balanced = ml.train_random_forest(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1)

Train accuracy: 0.9844097995545658


#### Gradient Boosting

In [11]:
gb_style_content_balanced = ml.train_gradient_boost(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1)

Train accuracy: 0.9974016332590943


#### Naive Bayes

In [12]:
nb_style_content_balanced = ml.train_naive_bayes(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1, remove_negatives=True)
nb_style_content_balanced, nb_style_content_balanced_scaler = nb_style_content_balanced['model'], nb_style_content_balanced['scaler']

Train accuracy: 0.9510022271714922


### Results

In [13]:
models = [lr_style_content_balanced, dt_style_content_balanced, rf_style_content_balanced, gb_style_content_balanced, nb_style_content_balanced]
names = ['Logistic Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting Tree', 'Naive Bayes']

In [14]:
results_style_content_balanced = ml.multi_model_results(models, names, style_content_test_balanced, style_test_balanced['target'], lr_style_content_balanced_scaler, nb_style_content_balanced_scaler)

In [15]:
results_style_content_balanced

,Accuracy,Precision,Recall,F1 Score,False Positive Rate,False Negative Rate,Area Under ROC Curve
Logistic Regression,0.965875,0.955182,0.979885,0.967376,0.049080,0.020115,0.965403
Decision Tree,0.951039,0.951289,0.954023,0.952654,0.052147,0.045977,0.950938
Random Forest,0.968843,0.957983,0.982759,0.970213,0.046012,0.017241,0.968373
Gradient Boosting Tree,0.967359,0.960452,0.977011,0.968661,0.042945,0.022989,0.967033
Naive Bayes,0.936202,0.957958,0.916667,0.936858,0.042945,0.083333,0.936861


## Imbalanced Dataset

In [16]:
style_content_train_imbalanced = pd.concat([word2vec_train_imbalanced['features'], style_train_imbalanced['features']], axis=1)
style_content_test_imbalanced = pd.concat([word2vec_test_imbalanced['features'], style_test_imbalanced['features']], axis=1)

### Train

#### Logistic Regression

In [17]:
lr_style_content_imbalanced = ml.train_logistic_regression(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1)
lr_style_content_imbalanced, lr_style_content_imbalanced_scaler = lr_style_content_imbalanced['model'], lr_style_content_imbalanced['scaler']

Train accuracy: 0.9927692931477227


#### Decision Tree

In [18]:
dt_style_content_imbalanced = ml.train_decision_tree(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1)

Train accuracy: 0.9833085552101636


#### Random Forest

In [19]:
rf_style_content_imbalanced = ml.train_random_forest(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1)

Train accuracy: 0.9846600892012434


#### Gradient Boosting

In [20]:
gb_style_content_imbalanced = ml.train_gradient_boost(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1)

Train accuracy: 0.9947965941343425


#### Naive Bayes

In [21]:
nb_style_content_imbalanced = ml.train_naive_bayes(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1, remove_negatives=True)
nb_style_content_imbalanced, nb_style_content_imbalanced_scaler = nb_style_content_imbalanced['model'], nb_style_content_imbalanced['scaler']

Train accuracy: 0.9080281119070145


### Results

In [22]:
models = [lr_style_content_imbalanced, dt_style_content_imbalanced, rf_style_content_imbalanced, gb_style_content_imbalanced, nb_style_content_imbalanced]
names = ['Logistic Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting Tree', 'Naive Bayes']

In [23]:
results_style_content_imbalanced = ml.multi_model_results(models, names, style_content_test_imbalanced, style_test_imbalanced['target'], lr_style_content_imbalanced_scaler, nb_style_content_imbalanced_scaler)

In [24]:
results_style_content_imbalanced

,Accuracy,Precision,Recall,F1 Score,False Positive Rate,False Negative Rate,Area Under ROC Curve
Logistic Regression,0.988108,0.940252,0.922840,0.931464,0.005628,0.077160,0.958606
Decision Tree,0.977838,0.903333,0.836420,0.868590,0.008590,0.163580,0.913915
Random Forest,0.979459,0.955882,0.802469,0.872483,0.003555,0.197531,0.899457
Gradient Boosting Tree,0.988649,0.946203,0.922840,0.934375,0.005036,0.077160,0.958902
Naive Bayes,0.912703,1.000000,0.003086,0.006154,0.000000,0.996914,0.501543


Comparing these with the content-only baseline, it is obvious that there is a at least a small improvement in the balanced dataset and a more significant improvement in the imbalanced dataset.<br>
It seems that the extra features are helpful in order to achieve better accuracy on the bigger dataset.

Another interesting observation is that the false negative rate of the two best performing algorithms is the same. This means that they both did not detect the same number of phishing emails (25).

In [25]:
lr_dt_predictions = ml.results_by_id([lr_style_content_imbalanced, gb_style_content_imbalanced], ['lr', 'gb'], pd.concat([style_test_imbalanced_complete[['email_id', 'email_class']],  style_content_test_imbalanced], axis=1), style_test_imbalanced_complete['email_id'], lr_style_content_imbalanced_scaler)
lr_dt_predictions[(lr_dt_predictions['True Class'] == True) & ((lr_dt_predictions['lr'] == False) & (lr_dt_predictions['gb'] == False))].shape[0]

15

15 of them were missclassified by both algorithms.

# Stacking

In machine learning, stacking refers to the proccess of using different learners (each one working best at learning a different part of the problem) called level 0 models as intermediate steps and then use their outputs to train another learner, called level 1 model. Thus, the final model is sometimes able to outperform the individual ones.

On this specific case, the different initial classifiers will be trained on both of the feature sets, and thus the final classifier essentially will combine information from both of them.

#### Final Classifiers

Only the three best classifiers will be used as a level 1 classifier, namely Logistic Regression (which is implemented by default), Random Forest and Gradient Boosting.

In [26]:
rf = ml.RandomForestClassifier(max_depth=5, n_estimators=20, random_state=ml.alg_random_state)
gb = ml.GradientBoostingClassifier(loss='log_loss', max_depth=3, learning_rate=0.1, random_state=ml.alg_random_state)

## Balanced Dataset

#### Train Initial Models

In [27]:
train_feature_sets_balanced = [{'name': 'style', 'features': style_train_balanced['features']}, {'name': 'word2vec', 'features': word2vec_train_balanced['features']}]
test_feature_sets_balanced = [{'name': 'style', 'features': style_test_balanced['features']}, {'name': 'word2vec', 'features': word2vec_test_balanced['features']}]

In [28]:
stacking_models_balanced = ml.train_models(train_feature_sets_balanced, style_train_balanced['target'])

### Single-algorithm

First, the stacking will be done only on the same algorithms with different feature sets, while also testing for different final_classifiers.

In [29]:
results_stacking_balanced_single = pd.DataFrame()

#### Logistic Regression

In [30]:
stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['dt', 'rf', 'gb', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'rf', 'gb', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['lr', 'rf', 'gb', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'rf', 'gb', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['lr', 'dt', 'gb', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'dt', 'gb', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['lr', 'dt', 'rf', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'dt', 'rf', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['lr', 'dt', 'rf', 'gb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'dt', 'rf', 'gb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

#### Random Forest

In [31]:
stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['dt', 'rf', 'gb', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'rf', 'gb', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['lr', 'rf', 'gb', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'rf', 'gb', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['lr', 'dt', 'gb', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'dt', 'gb', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['lr', 'dt', 'rf', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'dt', 'rf', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['lr', 'dt', 'rf', 'gb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'dt', 'rf', 'gb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

#### Gradient Boosting

In [32]:
stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['dt', 'rf', 'gb', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'rf', 'gb', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['lr', 'rf', 'gb', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'rf', 'gb', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['lr', 'dt', 'gb', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'dt', 'gb', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['lr', 'dt', 'rf', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'dt', 'rf', 'nb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['lr', 'dt', 'rf', 'gb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['lr', 'dt', 'rf', 'gb'])
results_stacking_balanced_single = pd.concat([results_stacking_balanced_single, stacked_preds['results']])

In [33]:
results_stacking_balanced_single

,Accuracy,Precision,Recall,F1 Score,False Positive Rate,False Negative Rate,Area Under ROC Curve
"Algorithms: lr, with LogisticRegression",0.965875,0.957746,0.977011,0.967283,0.046012,0.022989,0.965500
"Algorithms: dt, with LogisticRegression",0.956973,0.954416,0.962644,0.958512,0.049080,0.037356,0.956782
"Algorithms: rf, with LogisticRegression",0.970326,0.960674,0.982759,0.971591,0.042945,0.017241,0.969907
"Algorithms: gb, with LogisticRegression",0.974777,0.963585,0.988506,0.975887,0.039877,0.011494,0.974314
"Algorithms: nb, with LogisticRegression",0.940653,0.950292,0.933908,0.942029,0.052147,0.066092,0.940880
"Algorithms: lr, with RandomForestClassifier",0.968843,0.960563,0.979885,0.970128,0.042945,0.020115,0.968470
"Algorithms: dt, with RandomForestClassifier",0.962908,0.952381,0.977011,0.964539,0.052147,0.022989,0.962432
"Algorithms: rf, with RandomForestClassifier",0.968843,0.960563,0.979885,0.970128,0.042945,0.020115,0.968470
"Algorithms: gb, with RandomForestClassifier",0.976261,0.963687,0.991379,0.977337,0.039877,0.008621,0.975751
"Algorithms: nb, with RandomForestClassifier",0.940653,0.952941,0.931034,0.941860,0.049080,0.068966,0.940977


It is apparent that the results are better than the baseline models, and at some cases (when Random Forest of Gradient Boosting is used in one of the steps) even better than predicting with the merged feature sets.

The 6 best performing models will be kept to compare with other stacking configurations and the complete single-algorithm results dataset will be archived.

In [34]:
results_stacking_balanced_full = results_stacking_balanced_single.copy()

In [35]:
results_stacking_balanced_best = results_stacking_balanced_single.sort_values(by=['F1 Score'], ascending = [False]).head(6)

### Multi-algorithm

Of course, it is possible to also use the outputs of more than one classifier, on both feature sets.

In [36]:
results_stacking_balanced_multi = pd.DataFrame()

#### Logistic Regression

In [37]:
stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=[])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=[], result_row_name="Algorithms: all, with LogisticRegression")
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['dt', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['dt', 'nb', 'lr'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'lr'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['dt', 'nb', 'rf'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'rf'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['dt', 'nb', 'gb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'gb'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

#### Random Forest

In [38]:
stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=[])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=[], result_row_name="Algorithms: all, with RandomForestClassifier")
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['dt', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['dt', 'nb', 'lr'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'lr'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['dt', 'nb', 'rf'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'rf'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['dt', 'nb', 'gb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'gb'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

#### Gradient Boosting

In [39]:
stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=[])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=[], result_row_name="Algorithms: all, with GradientBoostingClassifier")
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['dt', 'nb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['dt', 'nb', 'lr'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'lr'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['dt', 'nb', 'rf'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'rf'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['dt', 'nb', 'gb'])
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'gb'])
results_stacking_balanced_multi = pd.concat([results_stacking_balanced_multi, stacked_preds['results']])

In [40]:
results_stacking_balanced_multi

,Accuracy,Precision,Recall,F1 Score,False Positive Rate,False Negative Rate,Area Under ROC Curve
"Algorithms: all, with LogisticRegression",0.976261,0.966292,0.988506,0.977273,0.036810,0.011494,0.975848
"Algorithms: gb, lr, rf, with LogisticRegression",0.976261,0.966292,0.988506,0.977273,0.036810,0.011494,0.975848
"Algorithms: gb, rf, with LogisticRegression",0.974777,0.963585,0.988506,0.975887,0.039877,0.011494,0.974314
"Algorithms: gb, lr, with LogisticRegression",0.976261,0.968927,0.985632,0.977208,0.033742,0.014368,0.975945
"Algorithms: lr, rf, with LogisticRegression",0.974777,0.963585,0.988506,0.975887,0.039877,0.011494,0.974314
"Algorithms: all, with RandomForestClassifier",0.974777,0.963585,0.988506,0.975887,0.039877,0.011494,0.974314
"Algorithms: gb, lr, rf, with RandomForestClassifier",0.977745,0.966387,0.991379,0.978723,0.036810,0.008621,0.977285
"Algorithms: gb, rf, with RandomForestClassifier",0.974777,0.963585,0.988506,0.975887,0.039877,0.011494,0.974314
"Algorithms: gb, lr, with RandomForestClassifier",0.976261,0.963687,0.991379,0.977337,0.039877,0.008621,0.975751
"Algorithms: lr, rf, with RandomForestClassifier",0.973294,0.966102,0.982759,0.974359,0.036810,0.017241,0.972974


As expected, using more than one classifier consistently improves the classification accuracy. The best level 1 classifier seems to be Gradient Boosting, and Logistic Regression gives better results when used as a level 0 classifier. However, all combinations performed quite well.

On the other hand, Naive Bayes and Decision Tree classifiers do not affect the result at all or even reduce the accuracy when used with the Random Forest Classifier, so from now on they will be excluded in order to reduce the execution time.

The top 8 models will be added to the best models dataset.

In [41]:
results_stacking_balanced_best = pd.concat([results_stacking_balanced_best, results_stacking_balanced_multi.sort_values(by=['F1 Score'], ascending = [False]).head(8)])

In [42]:
results_stacking_balanced_full = pd.concat([results_stacking_balanced_full, results_stacking_balanced_multi])

### Appending all features

Another variation of stacking includes appending the predictions to the other feature sets and then train the final classifier with all the features.<br>
Since using both feature sets has been proven to improve accuracy on the baselines, the predictions will be appended to the merged feature set.

In [43]:
results_stacking_balanced_append = pd.DataFrame()

#### Logistic Regression

In [44]:
stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['dt', 'nb', 'lr'], append_features=True)
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'lr'], append_features=True)
results_stacking_balanced_append = pd.concat([results_stacking_balanced_append, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['dt', 'nb', 'rf'], append_features=True)
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'rf'], append_features=True)
results_stacking_balanced_append = pd.concat([results_stacking_balanced_append, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], exclude_models=['dt', 'nb', 'gb'], append_features=True)
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'gb'], append_features=True)
results_stacking_balanced_append = pd.concat([results_stacking_balanced_append, stacked_preds['results']])

#### Random Forest

In [45]:
stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['dt', 'nb', 'lr'], append_features=True)
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'lr'], append_features=True)
results_stacking_balanced_append = pd.concat([results_stacking_balanced_append, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['dt', 'nb', 'rf'], append_features=True)
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'rf'], append_features=True)
results_stacking_balanced_append = pd.concat([results_stacking_balanced_append, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=rf, exclude_models=['dt', 'nb', 'gb'], append_features=True)
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'gb'], append_features=True)
results_stacking_balanced_append = pd.concat([results_stacking_balanced_append, stacked_preds['results']])

#### Gradient Boosting

In [46]:
stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['dt', 'nb', 'lr'], append_features=True)
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'lr'], append_features=True)
results_stacking_balanced_append = pd.concat([results_stacking_balanced_append, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['dt', 'nb', 'rf'], append_features=True)
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'rf'], append_features=True)
results_stacking_balanced_append = pd.concat([results_stacking_balanced_append, stacked_preds['results']])

stacked_clf = ml.train_stacked_models(stacking_models_balanced, train_feature_sets_balanced, style_train_balanced['target'], final_classifier=gb, exclude_models=['dt', 'nb', 'gb'], append_features=True)
stacked_preds = ml.test_stacked_models(stacking_models_balanced, test_feature_sets_balanced, style_test_balanced['target'], stacked_clf, exclude_models=['dt', 'nb', 'gb'], append_features=True)
results_stacking_balanced_append = pd.concat([results_stacking_balanced_append, stacked_preds['results']])

In [47]:
results_stacking_balanced_append

,Accuracy,Precision,Recall,F1 Score,False Positive Rate,False Negative Rate,Area Under ROC Curve
"Algorithms: gb, rf, with LogisticRegression (with appended features)",0.956973,0.951841,0.965517,0.958631,0.052147,0.034483,0.956685
"Algorithms: gb, lr, with LogisticRegression (with appended features)",0.951039,0.948718,0.956897,0.952790,0.055215,0.043103,0.950841
"Algorithms: lr, rf, with LogisticRegression (with appended features)",0.958457,0.946927,0.974138,0.960340,0.058282,0.025862,0.957928
"Algorithms: gb, rf, with RandomForestClassifier (with appended features)",0.968843,0.963173,0.977011,0.970043,0.039877,0.022989,0.968567
"Algorithms: gb, lr, with RandomForestClassifier (with appended features)",0.968843,0.963173,0.977011,0.970043,0.039877,0.022989,0.968567
"Algorithms: lr, rf, with RandomForestClassifier (with appended features)",0.973294,0.966102,0.982759,0.974359,0.036810,0.017241,0.972974
"Algorithms: gb, rf, with GradientBoostingClassifier (with appended features)",0.974777,0.961003,0.991379,0.975955,0.042945,0.008621,0.974217
"Algorithms: gb, lr, with GradientBoostingClassifier (with appended features)",0.980712,0.971831,0.991379,0.981508,0.030675,0.008621,0.980352
"Algorithms: lr, rf, with GradientBoostingClassifier (with appended features)",0.979228,0.971751,0.988506,0.980057,0.030675,0.011494,0.978915


Adding the initial feature sets to the final classifier seems to mostly harm performance on the balanced dataset. This is most likely due to overfitting, since the level 1 classifier becomes extremely specialized at recognizing the emails provided in the training set and fails to generalize for the test set.

However, when using Gradient Boosting as the final classifier, it manages to outperform the model without the appended features. This is no surprise, since Boosting methods in general are somewhat more resistant to overfitting.

The top 6 of these models will be added to the best result dataset, simply for comparison.

In [48]:
results_stacking_balanced_best = pd.concat([results_stacking_balanced_best, results_stacking_balanced_append.sort_values(by=['F1 Score'], ascending = [False]).head(6)])

In [49]:
results_stacking_balanced_full = pd.concat([results_stacking_balanced_full, results_stacking_balanced_append])

In [50]:
results_stacking_balanced_best.sort_values(by=['F1 Score'], ascending = [False]).head()

,Accuracy,Precision,Recall,F1 Score,False Positive Rate,False Negative Rate,Area Under ROC Curve
"Algorithms: gb, lr, with GradientBoostingClassifier (with appended features)",0.980712,0.971831,0.991379,0.981508,0.030675,0.008621,0.980352
"Algorithms: gb, lr, with GradientBoostingClassifier",0.979228,0.969101,0.991379,0.980114,0.033742,0.008621,0.978818
"Algorithms: lr, rf, with GradientBoostingClassifier (with appended features)",0.979228,0.971751,0.988506,0.980057,0.030675,0.011494,0.978915
"Algorithms: gb, lr, rf, with RandomForestClassifier",0.977745,0.966387,0.991379,0.978723,0.036810,0.008621,0.977285
"Algorithms: gb, lr, rf, with GradientBoostingClassifier",0.977745,0.966387,0.991379,0.978723,0.036810,0.008621,0.977285
